# Packages 

In [22]:
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import pandas as pd
from math import *
from sklearn.metrics import r2_score
import seaborn as sns
import warnings
warnings.filterwarnings("ignore") 
%matplotlib inline
from tqdm import tqdm

In [ ]:
output_path = '/Users/lanauaa/Desktop/Viral_dynamics_project/Data_replication_immunotron/'
experiment_name = '20230306_MHV_HeLa_kinetics'

# Import Data 

In [105]:
# PCR_data_raw = pd.read_csv('{}/{}/raw_data/....'.format(output_path,experiment_name)
PCR_data_raw = pd.read_csv('/Users/lanauaa/Desktop/TEST.csv')
                        
PCR_data = PCR_data_raw.dropna(axis=0, inplace=False)
PCR_data['letters']=np.nan
PCR_data['numbers']=np.nan
PCR_data['Housekeeping_gene']= np.nan
PCR_data['Gene_of_interest_1']=np.nan
PCR_data['DeltaCT']=np.nan
PCR_data['Delta2CT']=np.nan
PCR_data['Gene_of_interest_expression']=np.nan
PCR_data['Titer']=np.nan
PCR_data['Timepoint']=np.nan
                        
titers_list=[10,1,0.1,0.01,0.001,0.0001,0.00001,0]
timepoints_list=[24,20,16,12,8,5,4,3,2,1.5,1,0.5]

PCR_data


,Tag,CT,letters,numbers,Housekeeping_gene,Gene_of_interest_1,DeltaCT,Delta2CT,Gene_of_interest_expression,Titer,Timepoint
0,A1,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,A2,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,A3,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,A4,7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,A5,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
379,P20,8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
380,P21,8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
381,P22,8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
382,P23,8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [234]:
# PCR_data_raw = pd.read_csv('{}/{}/raw_data/....'.format(output_path,experiment_name)
PCR_data_raw = pd.read_csv('/Users/lanauaa/Desktop/TEST.csv')
                        
PCR_data = PCR_data_raw.dropna(axis=0, inplace=False)


                        
titers_list=[10,1,0.1,0.01,0.001,0.0001,0.00001,0]
timepoints_list=[24,20,16,12,8,5,4,3,2,1.5,1,0.5]


#define HK and GOI
PCR_data['GeneType']=['HK' if int(x[1:])<13 else 'GOI' for x in PCR_data['Tag']]
PCR_data = PCR_data.set_index(['GeneType'],append=True)

sub_df=PCR_data.query('GeneType=="HK"').reset_index().drop(['level_0'],axis=1)
sub_df['HK']=sub_df['CT']
sub_2df=PCR_data.query('GeneType=="GOI"').reset_index().drop(['level_0'],axis=1)
sub_df['GOI']=sub_2df['CT']
PCR_data=sub_df.drop(['CT','GeneType'],axis=1)

#set timepoints and titers
df_list=[]

PCR_data['numbers']=[int(x[1:]) for x in PCR_data['Tag']]
for i,time in enumerate(timepoints_list):
    i=i+1
    sub_df = PCR_data.query('numbers==@i')
    sub_df['Timepoint']=time
    df_list.append(sub_df)
PCR_data=pd.concat(df_list)

df2_list=[]
PCR_data['letters']=[x[0] for x in PCR_data['Tag']]
letters_list=PCR_data['letters'].unique()
for i,titer in enumerate(titers_list):
    j=letters_list[0+2*i]
    k=letters_list[1+2*i]
    sub_df = PCR_data.query('letters==@j')
    sub_df['Titer']= titer
    df2_list.append(sub_df)
    sub_df = PCR_data.query('letters==@k')
    sub_df['Titer']= titer
    df2_list.append(sub_df)

PCR_data=pd.concat(df2_list)



#delta2CT method
df3_list=[]
PCR_data['DeltaCT']=PCR_data['GOI']-PCR_data['HK']
for number in numbers_list:
    sub_df=PCR_data.query('numbers==@number')
    list=[sub_df.query('letters=="O"')['DeltaCT'],sub_df.query('letters=="P"')['DeltaCT']]
    delta=np.mean(list)
    sub_df['Delta2CT']=sub_df['DeltaCT']-delta
    df3_list.append(sub_df)
PCR_data=pd.concat(df3_list).drop(['numbers','letters'],axis=1)
PCR_data["GeneExpression"]=2**(-PCR_data['Delta2CT'])


PCR_data.drop(['Tag','HK','GOI','DeltaCT','Delta2CT'].to_csv('{}/{}/Data/'.format(output_path,experiment_name))

PCR_data

,Tag,HK,GOI,Timepoint,Titer,DeltaCT,Delta2CT,GeneExpression
0,A1,1,3,24.0,10.00000,2,3.5,8.838835e-02
12,B1,55,6,24.0,10.00000,-49,-47.5,1.990329e+14
24,C1,7,7,24.0,1.00000,0,1.5,3.535534e-01
36,D1,6,56,24.0,1.00000,50,51.5,3.140185e-16
48,E1,4,6,24.0,0.10000,2,3.5,8.838835e-02
...,...,...,...,...,...,...,...,...
143,L12,5,5,0.5,0.00010,0,-1.0,2.000000e+00
155,M12,4,7,0.5,0.00001,3,2.0,2.500000e-01
167,N12,4,7,0.5,0.00001,3,2.0,2.500000e-01
179,O12,8,9,0.5,0.00000,1,0.0,1.000000e+00
